# Analyse de données et modèle de prédiction

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

### Initialize Spark session

In [2]:
spark = SparkSession.builder \
    .appName("Vehicle Category Prediction") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
def print_df(df):
    print("Size", df.count())
    df.printSchema()
    df.show(5)

def load_data_in_csv_file(file_path):
    df = spark.read.options(delimiter=",", header=True, inferSchema=True).csv(file_path)

    # Display schema and first five rows of the DataFrame
    print("Schema and first rows in", file_path)
    print_df(df)
    
    return df

### Load the dataset

In [5]:
# Path du fichier dans Hadoop FS
hdfs_path = "hdfs:///tpa_groupe_14/data/fusion/fusion.csv"

# Charger CSV dans un DataFrame
vehicles_df = load_data_in_csv_file(hdfs_path)

Schema and first rows in hdfs:///tpa_groupe_14/data/fusion/fusion.csv
Size 100306
root
 |-- client_view.id: integer (nullable = true)
 |-- client_view.age: integer (nullable = true)
 |-- client_view.sexe: string (nullable = true)
 |-- client_view.taux: integer (nullable = true)
 |-- client_view.situationfamiliale: string (nullable = true)
 |-- client_view.nbenfantsacharge: integer (nullable = true)
 |-- client_view.deuxiemevoiture: integer (nullable = true)
 |-- immatriculation_co2_view.categorie: string (nullable = true)

+--------------+---------------+----------------+----------------+------------------------------+----------------------------+---------------------------+----------------------------------+
|client_view.id|client_view.age|client_view.sexe|client_view.taux|client_view.situationfamiliale|client_view.nbenfantsacharge|client_view.deuxiemevoiture|immatriculation_co2_view.categorie|
+--------------+---------------+----------------+----------------+-------------------------

### Data formatting

In [6]:
# Renommer les colonnes
column_names = vehicles_df.schema.names

for column_name in column_names:
    vehicles_df = vehicles_df.withColumnRenamed(column_name, column_name.replace(".", "_"))

print_df(vehicles_df)

Size 100306
root
 |-- client_view_id: integer (nullable = true)
 |-- client_view_age: integer (nullable = true)
 |-- client_view_sexe: string (nullable = true)
 |-- client_view_taux: integer (nullable = true)
 |-- client_view_situationfamiliale: string (nullable = true)
 |-- client_view_nbenfantsacharge: integer (nullable = true)
 |-- client_view_deuxiemevoiture: integer (nullable = true)
 |-- immatriculation_co2_view_categorie: string (nullable = true)

+--------------+---------------+----------------+----------------+------------------------------+----------------------------+---------------------------+----------------------------------+
|client_view_id|client_view_age|client_view_sexe|client_view_taux|client_view_situationfamiliale|client_view_nbenfantsacharge|client_view_deuxiemevoiture|immatriculation_co2_view_categorie|
+--------------+---------------+----------------+----------------+------------------------------+----------------------------+---------------------------+-------

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer

### Optional

In [ ]:
# Optional: Sub-sample the DataFrame
vehicles_df = vehicles_df.sample(withReplacement=False, fraction=0.3, seed=42)

In [8]:
# Preparer les features
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_index").fit(vehicles_df)
    for column in ["client_view_sexe", "client_view_situationfamiliale", "immatriculation_co2_view_categorie"]
]

assembler = VectorAssembler(
    inputCols=[
        "client_view_age", "client_view_taux", "client_view_nbenfantsacharge", "client_view_deuxiemevoiture",
        "client_view_sexe_index", "client_view_situationfamiliale_index"
    ],
    outputCol="features"
)

# Diviser le DataFrame en données training et test  (80% training, 20% test)
train_df, test_df = vehicles_df.randomSplit([0.8, 0.2], seed=42)

pipeline = Pipeline(stages=indexers + [assembler])
pipeline_model = pipeline.fit(train_df)

train_df = pipeline_model.transform(train_df)
test_df = pipeline_model.transform(test_df)

In [9]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [10]:
# Evaluation function
def evaluate_classifier(classifier, param_grid, train_df, test_df):
    evaluator = MulticlassClassificationEvaluator(labelCol="immatriculation_co2_view_categorie_index", predictionCol="prediction", metricName="accuracy")
    
    # Create the cross-validator
    cross_validator = CrossValidator(
        estimator=classifier,
        estimatorParamMaps=param_grid,
        evaluator=evaluator,
        numFolds=3, 
        seed=42
    )
    
    # Train the model with the best hyperparameters
    cv_model = cross_validator.fit(train_df)
    
    # Make predictions on the test data
    predictions = cv_model.transform(test_df)
    
    # Evaluate the model
    accuracy = evaluator.evaluate(predictions)
    
    return cv_model, accuracy

In [11]:
from pyspark.ml.classification import (
    DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, 
    LinearSVC, LogisticRegression, MultilayerPerceptronClassifier, NaiveBayes, OneVsRest
)

1. Random Forest

In [12]:
rf = RandomForestClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

# Define the hyperparameter grid
rf_param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

rf_model, rf_accuracy = evaluate_classifier(
    classifier=rf,
    param_grid=rf_param_grid,
    train_df=train_df,
    test_df=test_df
)

print("RandomForestClassifier Accuracy = {:.2f}".format(rf_accuracy))

24/06/08 21:34:53 WARN DAGScheduler: Broadcasting large task binary with size 1012.8 KiB
24/06/08 21:35:09 WARN DAGScheduler: Broadcasting large task binary with size 1248.3 KiB
24/06/08 21:35:10 WARN DAGScheduler: Broadcasting large task binary with size 1930.8 KiB
24/06/08 21:35:12 WARN DAGScheduler: Broadcasting large task binary with size 1110.3 KiB
24/06/08 21:35:22 WARN DAGScheduler: Broadcasting large task binary with size 1044.6 KiB
24/06/08 21:35:37 WARN DAGScheduler: Broadcasting large task binary with size 1310.3 KiB
24/06/08 21:35:38 WARN DAGScheduler: Broadcasting large task binary with size 2030.3 KiB
24/06/08 21:35:40 WARN DAGScheduler: Broadcasting large task binary with size 1110.9 KiB
24/06/08 21:35:51 WARN DAGScheduler: Broadcasting large task binary with size 1009.6 KiB
24/06/08 21:36:07 WARN DAGScheduler: Broadcasting large task binary with size 1277.8 KiB
24/06/08 21:36:09 WARN DAGScheduler: Broadcasting large task binary with size 1985.9 KiB
24/06/08 21:36:11 WAR

RandomForestClassifier Accuracy = 0.72


2. Decision Tree

In [12]:
dt = DecisionTreeClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

dt_param_grid = ParamGridBuilder()\
    .addGrid(dt.maxDepth, [5, 10])\
    .addGrid(dt.impurity, ["gini", "entropy"])\
    .build()

dt_model, dt_accuracy = evaluate_classifier(
    classifier=dt,
    param_grid=dt_param_grid,
    train_df=train_df,
    test_df=test_df
)

print("DecisionTreeClassifier Accuracy = {:.2f}".format(dt_accuracy))

DecisionTreeClassifier Accuracy = 0.72


3. Gradient Boosted Trees

In [13]:
gbt = GBTClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")
ovr = OneVsRest(classifier=gbt, labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

gbt_param_grid = ParamGridBuilder()\
    .addGrid(gbt.maxIter, [10, 20])\
    .addGrid(gbt.maxDepth, [5, 10])\
    .build()

gbt_model, gbt_accuracy = evaluate_classifier(
    classifier=ovr,
    param_grid=gbt_param_grid,
    train_df=train_df,
    test_df=test_df
)

print("GBTClassifier Accuracy = {:.2f}".format(gbt_accuracy))

24/06/03 15:46:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1026.2 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1024.5 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1025.0 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1025.6 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1026.8 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1029.0 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1033.7 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1042.9 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1060.7 KiB
24/06/03 15:46:40 WARN DAGScheduler: Broadcasting large task binary with size 1090.8 KiB
24/

GBTClassifier Accuracy = 0.72


4. Support Vector Machine

In [14]:
svm = LinearSVC(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")
ovr = OneVsRest(classifier=svm, labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

svm_param_grid = ParamGridBuilder()\
    .addGrid(svm.maxIter, [10, 20])\
    .addGrid(svm.regParam, [0.01, 0.1])\
    .build()

svm_model, svm_accuracy = evaluate_classifier(
    classifier=ovr,
    param_grid=svm_param_grid,
    train_df=train_df,
    test_df=test_df
)

print("LinearSVC Accuracy = {:.2f}".format(svm_accuracy))

LinearSVC Accuracy = 0.63


5. Logistic Regression

In [15]:
lr = LogisticRegression(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

lr_param_grid = ParamGridBuilder()\
    .addGrid(lr.maxIter, [10, 20])\
    .addGrid(lr.regParam, [0.01, 0.1])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

lr_model, lr_accuracy = evaluate_classifier(
    classifier=lr,
    param_grid=lr_param_grid,
    train_df=train_df,
    test_df=test_df
)

print("LogisticRegression Accuracy = {:.2f}".format(lr_accuracy))

LogisticRegression Accuracy = 0.65


6. Neural Networks (Multilayer Perceptron Classifier)

In [16]:
num_classes = train_df.select("immatriculation_co2_view_categorie_index").distinct().count()

mlp = MultilayerPerceptronClassifier(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features", layers=[6, 5, 4, num_classes])

mlp_param_grid = ParamGridBuilder()\
    .addGrid(mlp.maxIter, [50, 100])\
    .build()

mlp_model, mlp_accuracy = evaluate_classifier(
    classifier=mlp,
    param_grid=mlp_param_grid,
    train_df=train_df,
    test_df=test_df
)

print("MultilayerPerceptronClassifier Accuracy = {:.2f}".format(mlp_accuracy))

MultilayerPerceptronClassifier Accuracy = 0.56


7. Naive Bayes

In [17]:
nb = NaiveBayes(labelCol="immatriculation_co2_view_categorie_index", featuresCol="features")

nb_param_grid = ParamGridBuilder()\
    .addGrid(nb.smoothing, [0.5, 1.0, 1.5])\
    .build()

nb_model, nb_accuracy = evaluate_classifier(
    classifier=nb,
    param_grid=nb_param_grid,
    train_df=train_df,
    test_df=test_df
)

print("NaiveBayes Accuracy = {:.2f}".format(nb_accuracy))

NaiveBayes Accuracy = 0.45
